In [1]:
# Import dependencies 
import numpy as np
from twitter import *
import pandas as pd 
import datetime

In [2]:
#The tokens and consumers keys are my API authentication to use the Twitter API.  I'm using the twitter library as a python
#wrapper to call twitter request through python.
token = '1212463191910842368-l00ryMICeXQXXawl8w2zazEoFS7xDf'
token_secret = 'hbzjFRq1r2ygkydPXfrwrhm5zTa54F3mYEm2wi7Q09DgA'
consumer_key = 'g8V7jnF3dqMfg4LRhtobYB4Pl'
consumer_secret = 'Y0SHSL0H9X9gCtuy07mJ3cp144DS2JhwX4Uvgda2ph8NvIUswJ'

t = Twitter(
        auth=OAuth(token, token_secret, consumer_key, consumer_secret))

In [3]:
type(t)

twitter.api.Twitter

In [86]:
# The function below gets a person's timeline
# Note: Count can't be greater than 200
x = t.statuses.user_timeline(screen_name="BernieSanders", count=20, include_rts=False, tweet_mode = 'extended')

twitter.api.TwitterListResponse

In [5]:
# Creating a for loop to inserts Hillary's tweets in a list
Bernie_tweet = []
for x in x:
    Bernie_tweet.append(x)

In [6]:
Bernie_tweet

[{'created_at': 'Sat Jan 04 14:36:27 +0000 2020',
  'id': 1213469243632689152,
  'id_str': '1213469243632689152',
  'full_text': 'War and violence have got to be the last response, not the first response. https://t.co/xAd48XYro1',
  'truncated': False,
  'display_text_range': [0, 74],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1213468959829311489,
     'id_str': '1213468959829311489',
     'indices': [75, 98],
     'media_url': 'http://pbs.twimg.com/media/ENcbyWHW4AIltIp.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/ENcbyWHW4AIltIp.jpg',
     'url': 'https://t.co/xAd48XYro1',
     'display_url': 'pic.twitter.com/xAd48XYro1',
     'expanded_url': 'https://twitter.com/BernieSanders/status/1213469243632689152/video/1',
     'type': 'photo',
     'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
      'large': {'w': 640, 'h': 360, 'resize': 'fit'},
      'medium': {'w': 640, 'h': 360, 'resize': 'fit'},
 

In [75]:
type(Bernie_tweet)

list

In [8]:
# I just want to focus on ids and tweets
for element in Bernie_tweet:
    print (element['id'])
    print (element['full_text'])
    print('---')

1213469243632689152
War and violence have got to be the last response, not the first response. https://t.co/xAd48XYro1
---
1213289390857953280
I’ve gone to too many funerals in my own state. I’ve talked to too many mothers who’ve lost their kids in war. I’ve talked to too many soldiers who’ve come home with PTSD, who have come home without arms and without legs. https://t.co/b2AYbYPtRd
---
1213260657400827905
500,000 Americans are homeless today, including 30,000 veterans. 87 million are uninsured or underinsured. We must invest in the needs of the American people, not spend trillions on endless war. Join me in Decorah, Iowa: https://t.co/30JsCVqyq8
---
1213208074023882752
We’re just one month away from the Iowa caucus and now is the time to knock doors, make phone calls and get people involved. Join us live from our canvass launch in Waterloo, IA and sign up at https://t.co/nIMpJix9jw https://t.co/mvl0otRnaz
---
1213196007355428872
We must get our priorities right. We must invest in t

In [9]:
#(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

class TweetMiner(object):

    
    def __init__(self, t, result_limit = 200):
        
        self.t = t        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="BernieSanders", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
            else:
                statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets, tweet_mode = 'extended')
                statuses = [x for x in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [10]:
# Result limit == count parameter from our GetUserTimeline()
miner = TweetMiner(t, result_limit=200)

In [11]:
bernie = miner.mine_user_tweets(user="BernieSanders")
elizabeth = miner.mine_user_tweets(user="ewarren")

In [12]:
for x in range(5):
    print (bernie[x]['text'])
    print('---')

War and violence have got to be the last response, not the first response. https://t.co/xAd48XYro1
---
I’ve gone to too many funerals in my own state. I’ve talked to too many mothers who’ve lost their kids in war. I’ve talked to too many soldiers who’ve come home with PTSD, who have come home without arms and without legs. https://t.co/b2AYbYPtRd
---
500,000 Americans are homeless today, including 30,000 veterans. 87 million are uninsured or underinsured. We must invest in the needs of the American people, not spend trillions on endless war. Join me in Decorah, Iowa: https://t.co/30JsCVqyq8
---
We’re just one month away from the Iowa caucus and now is the time to knock doors, make phone calls and get people involved. Join us live from our canvass launch in Waterloo, IA and sign up at https://t.co/nIMpJix9jw https://t.co/mvl0otRnaz
---
We must get our priorities right. We must invest in the needs of America, not spend trillions more on endless wars.
---


In [13]:
for x in range(5):
    print (elizabeth[x]['text'])
    print('--')

Just a few hours into the new year, another trans American was killed in a senseless act of gun violence.

Dustin Parker was a father, a husband, and an advocate who looked out for his community. We must fight for justice for him and all trans Americans. https://t.co/3FzFythjUq
--
Nobody fighting for climate action should be afraid that they’re going to lose their job. In a Warren administration, I will fight to protect and strengthen workers’ voices, especially those who are holding their employers accountable. 
https://t.co/t9JmRuGoFn
--
Too many "American" companies have only one real loyalty: The short-term interests of their shareholders. If they can outsource jobs to save a nickel, that’s exactly what they'll do.

We have to defend good American jobs—and I've got a plan for that. https://t.co/UbLfugQ9uU
--
The catastrophic scenes from Australia's wildfires should alarm all of us. Climate change is driving even more dangerous and destructive fires across the world, from California

In [14]:
# A:
bernie_df = pd.DataFrame(bernie)
bernie_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sat Jan 04 14:36:27 +0000 2020,BernieSanders,2020-01-04 10:06:56.505073,418,War and violence have got to be the last respo...,1213469243632689152
1,Sat Jan 04 02:41:47 +0000 2020,BernieSanders,2020-01-04 10:06:56.505073,6967,I’ve gone to too many funerals in my own state...,1213289390857953280
2,Sat Jan 04 00:47:36 +0000 2020,BernieSanders,2020-01-04 10:06:56.505073,1850,"500,000 Americans are homeless today, includin...",1213260657400827905
3,Fri Jan 03 21:18:39 +0000 2020,BernieSanders,2020-01-04 10:06:56.505073,833,We’re just one month away from the Iowa caucus...,1213208074023882752
4,Fri Jan 03 20:30:42 +0000 2020,BernieSanders,2020-01-04 10:06:56.505073,5565,We must get our priorities right. We must inve...,1213196007355428872


In [15]:
# A:
miner = TweetMiner(t, result_limit=200)
elizabeth_tweets = miner.mine_user_tweets("ewarren", max_pages=30)

In [16]:
elizabeth_df = pd.DataFrame(elizabeth_tweets)

In [17]:
elizabeth_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sat Jan 04 01:10:41 +0000 2020,ewarren,2020-01-04 10:07:53.033834,1240,"Just a few hours into the new year, another tr...",1213266465018843136
1,Sat Jan 04 00:35:15 +0000 2020,ewarren,2020-01-04 10:07:53.033834,577,Nobody fighting for climate action should be a...,1213257550617026562
2,Fri Jan 03 23:39:22 +0000 2020,ewarren,2020-01-04 10:07:53.033834,363,"Too many ""American"" companies have only one re...",1213243486281064453
3,Fri Jan 03 22:49:43 +0000 2020,ewarren,2020-01-04 10:07:53.033834,1774,The catastrophic scenes from Australia's wildf...,1213230989067075590
4,Fri Jan 03 21:54:28 +0000 2020,ewarren,2020-01-04 10:07:53.033834,355,The Trump administration is expanding its dang...,1213217085448433664


In [18]:
bernie_df.shape

(2289, 6)

In [19]:
elizabeth_df.shape

(2927, 6)

In [20]:
# Joined elizabeth & Bernie Dataframe
tweets = pd.concat([elizabeth_df, bernie_df], axis=0)
tweets.shape

(5216, 6)

In [21]:
tweets.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Sat Jan 04 01:10:41 +0000 2020,ewarren,2020-01-04 10:07:53.033834,1240,"Just a few hours into the new year, another tr...",1213266465018843136
1,Sat Jan 04 00:35:15 +0000 2020,ewarren,2020-01-04 10:07:53.033834,577,Nobody fighting for climate action should be a...,1213257550617026562
2,Fri Jan 03 23:39:22 +0000 2020,ewarren,2020-01-04 10:07:53.033834,363,"Too many ""American"" companies have only one re...",1213243486281064453
3,Fri Jan 03 22:49:43 +0000 2020,ewarren,2020-01-04 10:07:53.033834,1774,The catastrophic scenes from Australia's wildf...,1213230989067075590
4,Fri Jan 03 21:54:28 +0000 2020,ewarren,2020-01-04 10:07:53.033834,355,The Trump administration is expanding its dang...,1213217085448433664


In [29]:
# The library below convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer

# The library stores dictionary values as counts 
from collections import Counter

# Using TfidfVectorizer to find ngrams for Elizabeth's tweets
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# Pulls all of Elizabeth's tweet text's into one giant string
summaries = "".join(elizabeth_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('big structural', 213),
 ('structural change', 203),
 ('big structural change', 198),
 ('ve got', 180),
 ('health care', 136),
 ('got plan', 122),
 ('ve got plan', 122),
 ('climate change', 89),
 ('gun violence', 88),
 ('donald trump', 83),
 ('trump administration', 74),
 ('grassroots movement', 73),
 ('ll fight', 69),
 ('make sure', 69),
 ('student loan', 63),
 ('fight hard', 60),
 ('climate crisis', 59),
 ('loan debt', 59),
 ('student loan debt', 58),
 ('plan https', 57)]

In [30]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(bernie_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('health care', 247),
 ('new deal', 103),
 ('green new', 99),
 ('green new deal', 99),
 ('donald trump', 88),
 ('fossil fuel', 85),
 ('working people', 79),
 ('white house', 79),
 ('young people', 64),
 ('health insurance', 62),
 ('student debt', 60),
 ('american people', 56),
 ('climate change', 56),
 ('billionaire class', 54),
 ('united states', 54),
 ('climate crisis', 53),
 ('wall street', 52),
 ('corporate greed', 52),
 ('working class', 48),
 ('join live', 46)]

In [99]:
music = t.search.tweets(q="#music", tweet_mode= 'extended')

In [100]:
type(music)

twitter.api.TwitterDictResponse

In [101]:
music

{'statuses': [{'created_at': 'Sat Jan 04 18:08:41 +0000 2020',
   'id': 1213522654440906753,
   'id_str': '1213522654440906753',
   'full_text': 'Always - YouTube #newmusic #newsongs #goodsongs #lovethissong #checkthisout #melodic #nowplaying #originalmusic #indiemusic #music #musician #singersongwriter #folk #contemporaryfolk  #folkrock #folkpop #folktronica #freakfolk #indiefolk #neofolk https://t.co/7kaOLjcQve',
   'truncated': False,
   'display_text_range': [0, 270],
   'entities': {'hashtags': [{'text': 'newmusic', 'indices': [17, 26]},
     {'text': 'newsongs', 'indices': [27, 36]},
     {'text': 'goodsongs', 'indices': [37, 47]},
     {'text': 'lovethissong', 'indices': [48, 61]},
     {'text': 'checkthisout', 'indices': [62, 75]},
     {'text': 'melodic', 'indices': [76, 84]},
     {'text': 'nowplaying', 'indices': [85, 96]},
     {'text': 'originalmusic', 'indices': [97, 111]},
     {'text': 'indiemusic', 'indices': [112, 123]},
     {'text': 'music', 'indices': [124, 130]},
